In [1]:
from __future__ import absolute_import, division, print_function

In [2]:
import pymysql
import numpy as np
from geoip import geolite2

In [5]:
db = pymysql.connect(host='localhost', user='root',passwd='leproject',db='ka_lite_central')

In [6]:
cur = db.cursor()

In [7]:
cur.execute("CREATE TABLE securesync_country (id serial PRIMARY KEY,\
    client_device_id varchar(32),\
    ip varchar(50),\
    countryname varchar(50),\
    timestamp datetime\
    );")

0

In [8]:
cur.execute("SELECT client_device_id, ip, timestamp FROM securesync_syncsession;")

1232969

In [9]:
results=cur.fetchall()

In [10]:
client_device_ids = [item[0] for item in results]
ips = [item[1] for item in results]
timestamps = [item[2] for item in results]

In [11]:
for client_device_id,ip,timestamp in zip(client_device_ids,ips,timestamps):
    try:
        geo = geolite2.lookup(ip.split(',')[0])
        if geo is not None:
            countryname = geo.country
        else: 
            countryname = 'XXXXX'
        cur.execute("INSERT INTO securesync_country (\
                    client_device_id,\
                    ip,\
                    countryname,\
                    timestamp\
                        )\
                    VALUES (\
                    %s,%s,%s,%s\
                    )",(client_device_id,ip,countryname,timestamp )\
                   )
    except ValueError:
        pass

In [12]:
db.commit()